# Processing Recipes into Data 

### Takes recipes from both BigOven and Spoonacular API and puts them into data formats useable in modeling. Needs to be rerun after pulling more recipes 

##### Includes all deduping



In [ ]:
import pandas as pd
import json
from pathlib import Path
import string
import re
import pickle
from itertools import chain
from fuzzywuzzy import fuzz, process
from nltk.corpus import stopwords 
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
from itertools import chain
from collections import Counter


In [69]:
#first load all recipe data
BOrecipes = []
BOingredients = []
BOids = []
BOinstructions = []
BOtitle = []
BOpath = Path('~/Dropbox/Not_work/insight/greenmarkets/bigoven/maindish/')
for i, po in enumerate(BOpath.expanduser().glob('*.json')):
    with po.open() as fh:
        try:
            r = json.load(fh)
        except Exception:
            print(str(po))
            continue
        BOrecipes.append(r)
        BOing = r.get('Ingredients')
        BOid = r.get('RecipeID')
        BOinstructions.append(r.get('Instructions'))
        BOtitle.append(r.get('Title'))
        BOids.append(r.get('RecipeID'))
        if BOing is not None:
            BOingredients.append([lemmatizer.lemmatize(re.sub(r'[^a-z ]', '', i['Name'].lower().strip()))
                                  for i in BOing if i is not None and i['Name'] is not None])
            


/Users/bridgitboulahanis/Dropbox/Not_work/insight/greenmarkets/bigoven/maindish/maindish-131-68-87332.json
/Users/bridgitboulahanis/Dropbox/Not_work/insight/greenmarkets/bigoven/maindish/maindish-131-71-87778.json
/Users/bridgitboulahanis/Dropbox/Not_work/insight/greenmarkets/bigoven/maindish/maindish-181-96-1158777.json
/Users/bridgitboulahanis/Dropbox/Not_work/insight/greenmarkets/bigoven/maindish/maindish-164-249-1283792.json
/Users/bridgitboulahanis/Dropbox/Not_work/insight/greenmarkets/bigoven/maindish/maindish-205-249-1797822.json
/Users/bridgitboulahanis/Dropbox/Not_work/insight/greenmarkets/bigoven/maindish/maindish-55-19-156227.json
/Users/bridgitboulahanis/Dropbox/Not_work/insight/greenmarkets/bigoven/maindish/maindish-127-74-2431270.json
/Users/bridgitboulahanis/Dropbox/Not_work/insight/greenmarkets/bigoven/maindish/maindish-112-159-862538.json
/Users/bridgitboulahanis/Dropbox/Not_work/insight/greenmarkets/bigoven/maindish/maindish-55-62-166860.json
/Users/bridgitboulahanis/

In [53]:
print('Current recipe count: ' + str(len(BOinstructions)))
print('Current list of ingredient count: ' + str(len(BOingredients)))
numing = []
numing.extend([len(r) for r in BOingredients])
print('Number of ingredients: ' + str(sum(numing)))
aa = list(chain.from_iterable(BOingredients))
print('Unique ingredients: ' + str(len(Counter(aa).keys())))


Current recipe count: 51473
Current list of ingredient count: 51473
Number of ingredients: 566025
Unique ingredients: 102008


In [56]:
#now we want to load spoonacular data and create a dataframe that has ingredients and their aisle 
# this will be our classification data

SArecipes = []
SAingredients = []
SAingclass = []
SAnoise = []
SAingID = []
SAmeta = []
SAmeasure = []

seenID = set()
SApath = Path('~/Dropbox/Not_work/insight/greenmarkets/spoonac/data/')
for i, po in enumerate(SApath.expanduser().glob('*/*.json')):
    with po.open() as fh:
        try:
            r = json.load(fh)
        except Exception:
            print(str(po))
            continue
        for rec in r:
            currenting = rec['extendedIngredients']
            for ing in currenting:
                SAnoise.extend(i.strip() for i in ing['metaInformation'])
                if ing['id'] in seenID:
                    continue
                seenID.add(ing['id']) 
                SAingredients.append(ing['name'])
                SAingclass.append(ing['aisle'])  
                SAingID.append(ing['id'])
                SAmeasure.append(ing['measures']['us']['unitLong'])
                SAmeta.extend([ing['meta']])

                    





In [57]:
#grab all the ingrdients from spoonacular and put in dataframe
classifying = pd.DataFrame({'ingredient': SAingredients, 'aisle': SAingclass, 'meta': SAmeta, 'measure': SAmeasure,  'id': SAingID})


In [58]:
#some words you noticed should get added to SAnoise
wta = ['ground', 'taste', 'ounce', 'large']
SAnoise.extend(wta)
SAnoise = set(SAnoise)
#remember you can check if it is in there with: 'fresh' in SAnoise

In [59]:
#suite of deduping functions

#on loading we tokenize, strip punctuation&digits&spaces and make lowercase 
#here we are removing meta words and stop words
stop_words = set(stopwords.words('english')) 

def removenoise(input_ing):
    assert type(input_ing) == list, type(input_ing)
    noise_free_ing = []
    for word in input_ing:
        checked = []
        splitit = word.split()
        checked.extend(i for i in splitit if i not in SAnoise and i not in stop_words)
        noise_free_ing.append(' '.join(checked))
    return noise_free_ing
        


In [60]:
#call the dedupe functions
nning = []
for i in BOingredients:
    nning.append(removenoise(i))



In [43]:
aa = list(chain.from_iterable(nning))
print('Unique deduped ingredients: ' + str(len(Counter(aa).keys())))

len(nning)

Unique deduped ingredients: 39912


35829

In [27]:
# classify ingredients into aisles
#RECALL! This is a very slow loop (~40 mins)
whichaisle = {}
seen = set()
notin = []
countclassified = 0
unclassified = 0
for i in nning:
    for j in i:
        if j in seen:
            continue
        seen.add(j) 
        highest = process.extractOne(j,classifying['ingredient']) #using FW to get the closest ingredient match
        a = classifying.index[classifying['ingredient'] == highest[0]] #pulling out the index of that ingredient in aisle df
        if highest[1] > 80: #set minimum bar for ingredient similarity
            whichaisle[j] = classifying['aisle'][a].tolist() # put in aisle classification
            countclassified += 1
        else:
            whichaisle[j]  = None
            notin.append(j)
            unclassified += 1
            #print(f'No match for {j}, closest was {highest[0]} at {highest[1]}')
    print(countclassified + unclassified)
        

15
25
34
39
47
52
63
72


83
87
89
96
98
107
115
126
133
138
145
151
154
158
161
166
167
174
178
182
186
191
193
199
210
217
229
231
239
247
254
259
260
263
270
272
285
289
294
299
304
308
310
313
323
328
332
335
340
347
350
352
355
359
363
366
371
375
381
388
390
393
395
396
400
401
404
404
408
411
414
417
430
433
439
441
454
459
464
470
472
476
478
485
488
491
494
499
505
511
514
516
521
524
527
530
531
533
539
541
543
549
553
554
558
561
563
565
568
571
574
577
580
586
587
597
599
601
609
612
615
617
626
631
633
635
638
648
650
652
655
657
658
661
666
666
667
672
674
679
682
689
696
698
703
705
708
713
715
716
717
718
721
726
728
732
735
740
741
743
746
746
749
754
755
755
758
760
764
764
766
768
772
773
777
782
786
788
791
795
799
802
802
803
804
805
807
809
813
815
818
821
823
827
829
830
835
836
843
847
847
850
854
857
858
859
864
869
872
874
876
880
884
884
885
886
889
890
894
900
904
907
909
911
911
912
915
919
921
922
927
928
930
931
933
934
935
938
940
941
945
946
948
953
955
959
960
962
962
967
968
9

3882
3882
3882
3882
3882
3882
3885
3885
3889
3890
3890
3891
3892
3892
3893
3894
3894
3896
3896
3896
3898
3900
3902
3902
3902
3902
3905
3905
3909
3912
3912
3914
3920
3921
3923
3923
3923
3924
3926
3926
3926
3926
3926
3927
3927
3930
3931
3931
3933
3934
3935
3949
3952
3954
3954
3954
3954
3959
3960
3961
3963
3967
3967
3967
3969
3969
3969
3972
3972
3974
3974
3977
3981
3984
3984
3985
3988
3992
3994
3994
3996
3999
3999
4001
4001
4001
4001
4002
4004
4005
4006
4007
4010
4010
4012
4012
4013
4015
4016
4016
4017
4020
4022
4022
4024
4025
4026
4028
4029
4029
4030
4031
4032
4034
4034
4035
4038
4039
4040
4042
4044
4044
4045
4046
4046
4055
4056
4058
4058
4061
4061
4062
4064
4064
4066
4068
4073
4076
4078
4080
4081
4082
4083
4084
4087
4089
4090
4090
4091
4092
4094
4095
4098
4098
4098
4100
4100
4102
4102
4103
4104
4105
4106
4109
4111
4117
4117
4119
4123
4135
4137
4139
4139
4139
4139
4139
4141
4147
4148
4148
4148
4149
4150
4150
4151
4153
4156
4156
4160
4160
4160
4160
4161
4162
4162
4164
4169
4169
4174
4175


6354
6355
6355
6356
6356
6357
6357
6359
6359
6359
6362
6363
6363
6365
6366
6366
6370
6371
6372
6377
6378
6380
6382
6382
6382
6382
6383
6383
6383
6387
6389
6390
6392
6393
6393
6396
6400
6400
6401
6402
6404
6404
6404
6404
6404
6408
6408
6410
6412
6412
6413
6417
6418
6419
6423
6423
6428
6430
6432
6433
6434
6438
6438
6439
6441
6441
6441
6441
6441
6441
6444
6444
6446
6446
6446
6447
6447
6448
6449
6450
6452
6453
6454
6456
6462
6466
6466
6470
6471
6478
6482
6483
6483
6483
6484
6486
6486
6487
6487
6487
6487
6488
6488
6489
6489
6489
6489
6492
6492
6492
6492
6492
6493
6493
6495
6495
6495
6496
6498
6500
6500
6501
6502
6504
6505
6505
6505
6505
6505
6505
6507
6507
6511
6512
6519
6519
6519
6519
6520
6521
6521
6522
6524
6525
6525
6526
6529
6532
6533
6534
6538
6538
6540
6540
6540
6545
6546
6546
6546
6549
6549
6555
6556
6557
6559
6559
6560
6564
6564
6565
6566
6568
6568
6568
6569
6569
6569
6569
6570
6571
6571
6572
6573
6573
6573
6575
6577
6577
6581
6583
6584
6584
6587
6590
6597
6597
6597
6598
6598
6598


8640
8641
8641
8643
8643
8645
8647
8648
8648
8648
8650
8652
8654
8655
8656
8658
8659
8660
8661
8661
8661
8662
8664
8664
8668
8668
8671
8674
8676
8676
8676
8677
8677
8677
8678
8680
8680
8681
8681
8681
8684
8684
8684
8686
8689
8695
8695
8696
8697
8697
8697
8700
8701
8701
8702
8703
8707
8709
8710
8711
8712
8715
8716
8720
8721
8721
8722
8723
8724
8726
8727
8728
8728
8732
8732
8732
8732
8733
8734
8734
8734
8735
8736
8738
8744
8744
8749
8751
8758
8758
8758
8762
8765
8765
8768
8768
8769
8769
8769
8771
8772
8773
8774
8774
8776
8780
8780
8781
8786
8787
8787
8788
8788
8788
8788
8789
8790
8790
8790
8791
8791
8796
8798
8799
8799
8800
8800
8801
8801
8803
8804
8804
8804
8804
8804
8805
8805
8805
8806
8808
8809
8809
8811
8814
8818
8818
8818
8818
8818
8818
8822
8824
8824
8825
8825
8828
8828
8829
8831
8831
8833
8834
8835
8835
8835
8835
8838
8840
8842
8842
8843
8843
8843
8843
8847
8848
8851
8851
8851
8852
8853
8857
8860
8867
8867
8867
8869
8870
8872
8879
8883
8884
8884
8885
8887
8889
8891
8892
8893
8893


...

30923
30923
30923
30931
30933
30934
30935
30936
30938
30938
30940
30940
30941
30941
30941
30941
30941
30941
30943
30943
30944
30944
30944
30944
30950
30950
30951
30951
30951
30951
30951
30951
30958
30959
30959
30959
30959
30959
30959
30959
30960
30960
30960
30960
30961
30961
30966
30967
30967
30969
30970
30971
30972
30975
30975
30975
30977
30977
30978
30978
30978
30978
30978
30978
30978
30978
30978
30979
30983
30983
30983
30983
30986
30988
30989
30994
30995
30996
30996
30996
30997
30997
30997
30998
30999
31001
31003
31003
31004
31005
31006
31008
31008
31009
31009
31009
31009
31011
31011
31012
31012
31012
31012
31013
31013
31013
31013
31015
31016
31016
31016
31016
31017
31017
31018
31018
31018
31020
31020
31020
31023
31023
31024
31025
31025
31026
31030
31030
31030
31030
31030
31030
31031
31032
31033
31033
31035
31035
31036
31036
31036
31036
31037
31037
31038
31038
31043
31044
31044
31045
31046
31047
31050
31051
31051
31051
31051
31052
31052
31052
31052
31052
31053
31053
31056
31056
3105

32166
32169
32169
32169
32169
32169
32171
32171
32171
32171
32171
32171
32171
32172
32172
32173
32173
32175
32176
32176
32177
32177
32177
32177
32177
32177
32178
32178
32179
32180
32180
32180
32180
32182
32182
32182
32182
32183
32183
32183
32186
32186
32190
32191
32199
32200
32203
32203
32204
32204
32206
32207
32209
32211
32211
32213
32213
32214
32214
32214
32215
32216
32216
32218
32218
32218
32220
32221
32221
32221
32222
32222
32222
32222
32225
32229
32229
32230
32230
32230
32230
32232
32232
32232
32233
32234
32236
32236
32236
32237
32237
32237
32237
32238
32238
32240
32241
32241
32244
32244
32245
32245
32246
32246
32246
32246
32246
32249
32249
32250
32253
32253
32253
32254
32255
32257
32257
32257
32258
32259
32259
32259
32260
32260
32261
32261
32262
32263
32263
32264
32264
32264
32264
32265
32265
32267
32268
32268
32271
32276
32276
32280
32280
32282
32283
32283
32283
32284
32284
32285
32285
32285
32285
32285
32286
32286
32286
32287
32288
32288
32288
32288
32288
32288
32291
32291
3229

33340
33340
33340
33341
33342
33342
33342
33342
33342
33342
33343
33344
33345
33345
33346
33346
33347
33348
33348
33348
33351
33351
33351
33351
33351
33353
33353
33358
33359
33362
33362
33362
33362
33363
33363
33363
33364
33365
33365
33366
33366
33366
33366
33369
33369
33370
33371
33372
33373
33373
33373
33373
33374
33375
33375
33377
33377
33378
33378
33378
33380
33380
33381
33382
33384
33384
33387
33388
33389
33391
33391
33393
33395
33396
33396
33397
33397
33397
33397
33397
33399
33399
33399
33401
33401
33401
33401
33401
33402
33402
33403
33403
33403
33403
33404
33406
33411
33412
33412
33415
33415
33415
33415
33418
33419
33420
33434
33434
33434
33434
33434
33434
33435
33436
33436
33436
33437
33437
33437
33438
33439
33439
33445
33445
33446
33447
33447
33447
33447
33448
33448
33448
33450
33451
33455
33455
33456
33459
33460
33460
33461
33465
33465
33465
33465
33465
33466
33466
33467
33467
33467
33467
33467
33467
33469
33469
33469
33469
33470
33471
33471
33472
33473
33475
33476
33476
3347

34465
34465
34466
34466
34466
34466
34466
34466
34466
34466
34470
34471
34471
34473
34473
34473
34473
34474
34474
34474
34475
34476
34476
34476
34477
34477
34477
34477
34478
34478
34479
34479
34480
34480
34481
34482
34482
34482
34483
34483
34483
34483
34483
34483
34487
34488
34489
34490
34491
34492
34492
34492
34492
34492
34493
34493
34494
34496
34496
34497
34497
34500
34510
34511
34511
34511
34511
34511
34512
34513
34514
34516
34517
34517
34520
34524
34524
34524
34525
34528
34529
34529
34534
34535
34538
34539
34539
34539
34539
34540
34544
34545
34545
34545
34546
34546
34548
34550
34550
34552
34554
34554
34554
34555
34556
34556
34557
34557
34557
34557
34559
34560
34560
34561
34562
34562
34564
34566
34566
34567
34569
34570
34571
34571
34572
34573
34574
34574
34574
34574
34574
34574
34574
34575
34577
34578
34580
34580
34580
34581
34581
34581
34582
34586
34586
34586
34587
34589
34590
34590
34590
34592
34592
34593
34593
34593
34594
34595
34597
34597
34597
34597
34598
34598
34599
34600
3460

35618
35620
35620
35622
35622
35622
35622
35622
35622
35622
35631
35631
35631
35631
35631
35632
35633
35634
35634
35634
35634
35638
35639
35639
35639
35639
35642
35644
35644
35644
35645
35647
35648
35648
35648
35649
35649
35650
35651
35651
35655
35655
35656
35656
35658
35659
35659
35660
35660
35660
35660
35661
35661
35661
35661
35661
35662
35663
35665
35665
35671
35672
35673
35673
35673
35674
35674
35675
35675
35675
35675
35676
35676
35676
35676
35676
35676
35680
35680
35680
35680
35680
35680
35680
35680
35680
35680
35682
35682
35682
35683
35685
35685
35685
35687
35687
35688
35688
35688
35688
35688
35689
35691
35693
35694
35695
35696
35696
35698
35698
35698
35698
35698
35698
35698
35699
35699
35700
35702
35702
35705
35705
35705
35707
35707
35709
35710
35712
35712
35713
35715
35715
35716
35716
35717
35717
35717
35719
35721
35723
35724
35725
35725
35725
35727
35727
35728
35729
35729
35729
35729
35729
35730
35732
35732
35732
35732
35733
35733
35733
35734
35741
35743
35743
35750
35750
3575

In [61]:
#output the data -- right now just the ingredients and classifier, maybe later the whole list?
pickle.dump(nning,open("generated_data/cleaned_ingredients.pkl","wb"))
pickle.dump(whichaisle,open("generated_data/ingredient_aisle.pkl","wb"))
pickle.dump(BOinstructions, open("generated_data/instructions.pkl", "wb"))
pickle.dump(BOtitle, open("generated_data/recipetitle.pkl", "wb"))


In [62]:
len(nning)

51473

In [46]:
len(whichaisle)

36657

In [47]:
len(BOinstructions)

35829